In [3]:
import Functions as F 
import pandas as pd 
import numpy as np 
%matplotlib qt 
results = "results/results05.csv"
nr_agents = 6
nr = 21
Viz = F.VisualizeAngle(results, nr_agents)
Viz.get_coor()
Viz.plot_angle_in_plane(nr, True)

Dij = F.Dijkstras(results, nr_agents, nr, Viz.R)
Dij.run_dijkstra(6, 6)

## Plot path on top of image

In [4]:
import SimpleITK as sitk 
import numpy as np 
import os 
import matplotlib.pyplot as plt 

Facial paths 

In [26]:
facial_paths = os.listdir("paths/paths_facial")
itkimage = sitk.ReadImage("Cmarl/src/data/images/Data_Scaled/ear1-P1051_2.nii.gz")

mask = sitk.GetArrayFromImage(sitk.ReadImage("paths/paths_facial/Landmark_ear1-P1051_2.nii.gz"))
im = sitk.GetArrayFromImage(itkimage)


In [ ]:
import statistics
x, y, z = np.where(mask == 1)
slice = statistics.mode(x)
slice = 68 

plt.imshow(im[:, slice], cmap = "gray")
plt.scatter(z[y == slice], x[y == slice], color ="r", s = 2)
plt.title("FN Path (Axial view)")
plt.axis("off")
plt.show()

Chorda paths 

In [5]:
chorda_paths = os.listdir("paths/paths_chorda")
itkimage = sitk.ReadImage("Cmarl/src/data/images/Data_good_Scaled/" + chorda_paths[0][6:])

mask = sitk.GetArrayFromImage(sitk.ReadImage("paths/paths_chorda/" + chorda_paths[0]))
im = sitk.GetArrayFromImage(itkimage)


In [18]:
import statistics
x, y, z = np.where(mask == 1)
slice = statistics.mode(z)

plt.imshow(im[0:70, 0:70, slice], cmap = "gray")
plt.scatter(y[z == slice], x[z == slice], color ="r", s = 2)
plt.title("CTY Path (Sagittal view)")
plt.axis("off")
plt.show()

In [8]:
print([x, y, z])

[array([37, 38, 39, 40, 40, 40, 40, 40, 40], dtype=int64), array([56, 56, 56, 56, 56, 56, 56, 56, 56], dtype=int64), array([30, 30, 30, 25, 26, 27, 28, 29, 30], dtype=int64)]
